# PyInstaller
### Create frozen stand alone executables under Windows, Linux, or OSX
@ftlphys
Michael Gilbert

### What is Pyinstaller and what does it seek to accomplish?

*PyInstaller is a program that freezes (packages) Python programs into stand-alone executables, under Windows, Linux, Mac OS X, FreeBSD, Solaris and AIX. Its main advantages over similar tools are that PyInstaller works with Python 2.7 and 3.3—3.5, it builds smaller executables thanks to transparent compression, it is fully multi-platform, and use the OS support to load the dynamic libraries, thus ensuring full compatibility.*  

*The main goal of PyInstaller is to be compatible with 3rd-party packages out-of-the-box.* - [Pyinstaller](http://www.pyinstaller.org/)

### So does it accomplish this?

Mostly!  But it is under very heavy and renewed development! [Github Pyinstaller Repository](https://github.com/pyinstaller/pyinstaller)

### How it works...

* pip install pyinstaller
* pyinstaller main.py (from the commandline)

* pyi-makespec : makes a spec file
    - specify data files, .dll, or .so files for bundling
    - specify run time options
* pyi-bindepend : inspect executable files to determine dependencies

### Third party library compatability  
#### [List of supported packages](https://github.com/pyinstaller/pyinstaller/wiki/Supported-Packages)
#### [Note about multiprocessing...](https://github.com/pyinstaller/pyinstaller/wiki/Recipe-Multiprocessing)

#### Pyinstaller hooks
* Allows for hidden imports to be added to the binary
    - This helps with peculiar dynamic imports, like assigned sys.modules
* Allows for the inclusion of required binaries
* Added via short python scripts
    

### Just a few options...

* Windowed (no console) –w : This will compile you program to launch without a command prompt window opening
* Single File –F : This will compile your executable into a single file; whereas, the default is to compile into a single directory.
* UPX option : Allows for extra compression of the executable This is provided by a separate library that you can obtain online, http://upx.sourceforge.net/ .
* Clean option --clean : This will clean up some build specific files
* Debug option --debug : This will output extra information to the command prompt about the run process when the executable is launched, good for debugging runtime issues.

### Want a GUI interface?

In [2]:
import wx
%gui wx
from subprocess import Popen
import os
import threading

myEVT_COMP = wx.NewEventType()
EVT_COMP = wx.PyEventBinder(myEVT_COMP, 1)


class MyFrame(wx.Frame):
    def __init__(self, *args, **kwds):
        wx.Frame.__init__(self, *args, **kwds)
        self.panel_1 = wx.Panel(self, -1)
        self.label_1 = wx.StaticText(self.panel_1, -1,
                                     "Select Main Project File: ")
        self.text_ctrl_1 = wx.TextCtrl(self.panel_1, -1, "")
        self.button_2 = wx.Button(self.panel_1, -1, "Browse")
        self.button_1 = wx.Button(self.panel_1, -1, "Compile")

        optionslist = ['Use Anaconda', 'Windowed (no console) -w', 'Single File -F',
                       'UPX option', 'Clean option', 'Debug option']
        lb = wx.CheckListBox(self.panel_1, -1, (80, 50), wx.DefaultSize,
                             optionslist)
        self.Bind(wx.EVT_CHECKLISTBOX, self.EvtCheckListoptions, lb)
        lb.SetSelection(0)
        self.lb = lb
        self.checked = []

        self.exec_dir = os.getcwd()

        self.Bind(wx.EVT_BUTTON, self.compileit, self.button_1)
        self.Bind(wx.EVT_BUTTON, self.browseit, self.button_2)
        self.Bind(wx.EVT_TEXT, self.EvtText, self.text_ctrl_1)
        self.Bind(EVT_COMP, self.complete_compile)

        self.__set_properties()
        self.__do_layout()

    def __set_properties(self):
        self.SetTitle("Python Executable Compiler")
        self.label_1.SetFont(wx.Font(10, wx.DEFAULT, wx.NORMAL, wx.NORMAL,
                                     0, ""))

    def __do_layout(self):
        sizer_1 = wx.BoxSizer(wx.VERTICAL)
        sizer_2 = wx.BoxSizer(wx.VERTICAL)
        sizer_3 = wx.BoxSizer(wx.HORIZONTAL)
        sizer_3.Add(self.label_1, 0, wx.RIGHT | wx.TOP | wx.BOTTOM |
                    wx.LEFT | wx.ALIGN_CENTER_VERTICAL, 5)
        sizer_3.Add(self.text_ctrl_1, 0, wx.TOP | wx.BOTTOM |
                    wx.ALIGN_CENTER_HORIZONTAL | wx.ALIGN_CENTER_VERTICAL, 5)
        sizer_3.Add(self.button_2, 0, wx.RIGHT | wx.LEFT | wx.TOP |
                    wx.BOTTOM | wx.ALIGN_CENTER_VERTICAL, 5)
        sizer_2.Add(sizer_3, 1, wx.EXPAND, 0)
        sizer_2.Add(self.lb, 1, wx.ALIGN_CENTER_HORIZONTAL, 0)
        sizer_2.Add(self.button_1, 0, wx.TOP | wx.BOTTOM |
                    wx.ALIGN_CENTER_HORIZONTAL, 10)
        self.panel_1.SetSizer(sizer_2)
        sizer_1.Add(self.panel_1, 1, wx.EXPAND, 0)
        self.SetSizer(sizer_1)
        sizer_1.Fit(self)
        self.Layout()

    def EvtCheckListoptions(self, event):
        index = event.GetSelection()
        label = self.lb.GetString(index)
        if self.lb.IsChecked(index):
            self.checked.append(label)
        elif label in self.checked:
            self.checked.remove(label)
        # so that (un)checking also selects (moves the highlight)
        self.lb.SetSelection(index)

    def EvtText(self, event):
        self.project = event.GetString()

    def compileit(self, evt):
        self.button_1.Disable()
        self.compiling = CompileThread(self, self.pyfile, self.checked)
        self.compiling.start()

    def complete_compile(self, evt):
        self.compiling.join()
        self.button_1.Enable()

    def browseit(self, evt):
        """
        Create and show the Open FileDialog
        """
        dlg = wx.FileDialog(
            self, message="Choose a file",
            defaultFile="",
            wildcard="Python Files (*.py)|*.py",
            style=wx.OPEN | wx.CHANGE_DIR
            )
        if dlg.ShowModal() == wx.ID_OK:
            self.pyfile = dlg.GetPaths()[0]
            self.text_ctrl_1.SetValue(self.pyfile)
        dlg.Destroy()


class GeneratedCOMP(wx.PyCommandEvent):

    def __init__(self, etype, eid):
        wx.PyCommandEvent.__init__(self, etype, eid)


class CompileThread(threading.Thread):
    def __init__(self, parent, pyfile, checked):
        self._parent = parent
        self.pyfile = pyfile
        self.checked = checked
        threading.Thread.__init__(self)

    def run(self):
        tempbatch = open('compile_batch.bat', 'w')
        tempbatch.write(r'@echo off')
        tempbatch.write('\n')
        tempbatch.write(r'cd '+os.path.dirname(self.pyfile))
        tempbatch.write('\n')
        if 'Use Anaconda' in self.checked:
            tempbatch.write('call ' + os.path.join(os.getenv('HOMEDRIVE'), os.getenv('HOMEPATH'),
                            'AppData\\Local\\Continuum\\Anaconda\\Scripts\\anaconda.bat'))
            tempbatch.write('\n')
        if 'Windowed (no console) -w' not in self.checked and 'Single File -F' not in self.checked:
            options = '-D'
        elif 'Windowed (no console) -w' in self.checked and 'Single File -F' in self.checked:
            options = '-F -w'
        elif 'Windowed (no console) -w' in self.checked and 'Single File -F' not in self.checked:
            options = '-D -w'
        elif 'Windowed (no console) -w' not in self.checked and 'Single File -F' in self.checked:
            options = '-F'
        if 'UPX option' in self.checked:
            options += ' --upx-dir='+ os.path.join(self._parent.exec_dir, 'upx') + ' '
        if 'Clean option' in self.checked:
            options = '--clean ' + options
        if 'Debug option' in self.checked:
            options = options + ' --debug'
        tempbatch.write(r'pyinstaller ' + options + ' ' + os.path.basename(self.pyfile))
        tempbatch.close()

        p = Popen("compile_batch.bat", cwd=os.path.dirname(self.pyfile))
        p.communicate()
        print "complete!"
        evt = GeneratedCOMP(myEVT_COMP, -1)
        wx.PostEvent(self._parent, evt)

# app = wx.App(0)
frame_1 = MyFrame(None, -1, "")
# app.SetTopWindow(frame_1)
frame_1.Show()
# app.MainLoop()

True